In [1]:
import pandas as pd
import glob as gb
import os

files = gb.glob('./data/Ward_Age/*')
dataname = '町丁目名年齢年代別人口分布'

d=os.getcwd()
prefix = d + '/../data/Ward_Age/'


# c
#g_code = pd.read_csv(prefix+'東京都人口世帯数_estat.txt', encoding='SHIFT-JIS', dtype={'KEY_CODE':str})
#g_code[g_code['HYOSYO']>3]
#g_code=g_code[['KEY_CODE','NAME','CITYNAME']]
wds = ['市区町村コード','市区町村名','大字町丁目コード','大字町丁目名']
c_code = pd.read_csv(prefix + '13_2020.csv', encoding='utf-8', dtype={'大字町丁目コード':str})[wds]

## 1. 大田区

In [2]:
ohta_file = prefix + '大田区2022年1月_rev.xlsx'

ohta_data = pd.read_excel(ohta_file).set_index('年齢')

ohta_finedata = pd.DataFrame(ohta_data.stack()).reset_index().set_axis(['年齢','町丁目名','人口'],axis=1)

ohta_finedata = ohta_finedata[['町丁目名','年齢','人口']]

ohta_finedata['町丁目名'] = ohta_finedata['町丁目名'].str.replace('　','')

ohta_finedata = pd.merge(ohta_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

ohta_finedata['区']='大田'

In [3]:
#ohta_finedata.to_excel(prefix + dataname + '（大田）.xlsx', index=False)

bin_band =  list(range(0,131,5))
ew=pd.cut(ohta_finedata['年齢'], bin_band, right=False)
#pd.DataFrame(pd.DataFrame(ew).value_counts()).sort_values('年齢')

## 2. 品川

In [4]:
male=list(range(0,241,2))
female=list(range(1,241,2))

shina_file = prefix + '品川区2023年1月_rev.xls'
shina_data = pd.read_excel(shina_file, dtype={'行政コード':int})

shina_m = shina_data.loc[:,['行政コード','丁目名']+male].set_index(['行政コード','丁目名']).stack()
shina_f = shina_data.loc[:,['行政コード','丁目名']+female].set_index(['行政コード','丁目名']).stack()

shina_m_fine= pd.DataFrame(shina_m).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_m_fine['年齢']=shina_m_fine['age']//2

shina_f_fine= pd.DataFrame(shina_f).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_f_fine['年齢']=(shina_f_fine['age'])//2

scols=['丁目名','年齢','人口']
mer=['丁目名','年齢']

shina_m_fine=shina_m_fine.drop(columns=['age'])[scols]
shina_f_fine=shina_f_fine.drop(columns=['age'])[scols]


shina_finedata = pd.merge(shina_m_fine, shina_f_fine, on=mer)

#for t,i in shina_finedata.iterrows():
#    shina_finedata.loc[i,'丁目名'] = shina_finedata.loc[i, '丁目名'].replace('\u3000','')
#    print(shina_finedata.loc[i, '丁目名'])

shina_finedata['人口'] = shina_finedata['人口_x']+shina_finedata['人口_y']
shina_finedata = shina_finedata.drop(columns=['人口_x','人口_y']).reset_index(drop=True)
shina_finedata = shina_finedata.rename(columns={'丁目名':'町丁目名'})
shina_finedata = pd.merge(shina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

shina_finedata['区']='品川'

In [5]:
shina_finedata

,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区
0,北品川１丁目,0,20,13109,品川区,131090019001,北品川１丁目,品川
1,北品川１丁目,1,17,13109,品川区,131090019001,北品川１丁目,品川
2,北品川１丁目,2,21,13109,品川区,131090019001,北品川１丁目,品川
3,北品川１丁目,3,18,13109,品川区,131090019001,北品川１丁目,品川
4,北品川１丁目,4,14,13109,品川区,131090019001,北品川１丁目,品川
...,...,...,...,...,...,...,...,...
16195,東八潮,115,0,13109,品川区,131090005000,東八潮,品川
16196,東八潮,116,0,13109,品川区,131090005000,東八潮,品川
16197,東八潮,117,0,13109,品川区,131090005000,東八潮,品川
16198,東八潮,118,0,13109,品川区,131090005000,東八潮,品川


In [6]:
#shina_finedata.to_excel(prefix + dataname + '（品川）.xlsx', index=False)

## 3. 目黒

In [7]:
meg_file = prefix +'目黒区2022年12月_rev.xlsx'
meg_data = pd.read_excel(meg_file)
#meg_data

top = meg_data[meg_data['a']=='町丁別年齢別人口表'].reset_index()['index']
end = meg_data[meg_data['a']=='104以上'].reset_index()['index']
#print(list(top))
#print(list(end))

meg_tmp = pd.DataFrame()

#meg_data['町丁目名']=''

for k in top:
    chomei = meg_data.iloc[k+1,0].replace('　','')

    for j in list(range(k+4,k+129)):
        meg_data.loc[j,'町丁目名'] = chomei

    chodf = meg_data.iloc[(k+4):(k+129),:]
    
    chodf = chodf[~(chodf['a'].str.contains('～',na=False))]
    chodf = chodf[~(chodf['a'].str.contains('以上',na=False))]
    
    chodf = chodf.rename(columns={'a':'年齢', 'b':'人口'})
    chodf['年齢'] = chodf['年齢'].astype(int)
    
    meg_tmp = pd.concat([meg_tmp, chodf])

meg_finedata = meg_tmp[['町丁目名','年齢','人口']]
meg_finedata = pd.merge(meg_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

meg_finedata['区']='目黒'

In [8]:
#meg_finedata.to_excel(prefix + dataname + '（目黒）.xlsx', index=False)

## 4. 港区

In [9]:
import tabula

mina_file = prefix + '港区2022年4月.pdf'

mina_data = tabula.read_pdf(mina_file,pages="1-117",stream=True)

In [10]:
textfile = open(prefix+'港区町丁目名.txt','r')
output_text = textfile.read()
textfile.close()

import re

pattern = '(.+丁目)|(麻布.+町)|(麻布狸穴町)$'

minacho = re.findall(pattern, output_text)

from itertools import chain
def flatten(nested_list):
    return list(chain.from_iterable(nested_list))

#flatten(list(map(lambda x: list(x), minacho)))

chomeilist = list(map((lambda y: ''.join(y)), list(map(lambda x: list(x), minacho))))

In [11]:
mina_finedata = pd.DataFrame()

for i in range(0,116):
    if i != 39:
        k = mina_data[i].iloc[:21,0:3].set_axis(['世代','a','b'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k = k.fillna(0)
        k = k.astype({'a':str, 'b':str})
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)
        k['b'] = k['b'].str.replace(',','')
        k['b'] = k['b'].str.replace(r'0\.0','0', regex=True)
    else:
        k = mina_data[i].iloc[:21,0:2].set_axis(['世代','a'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)        
        k['b'] = '0'
        
    mina_finedata = pd.concat([mina_finedata, k])

mina_finedata = mina_finedata.astype({'a':int, 'b':int}).reset_index(drop=True) 

mina_finedata['人口'] = mina_finedata['a'] + mina_finedata['b']
#

for r in mina_finedata.itertuples():
    if not(((r.a == 0) and (r.b==r.人口)) or ((r.b==0) and (r.a==r.人口))):
        print(r.町丁目名, ' ', r.a, r.b)

mina_finedata = mina_finedata.drop(columns=['a','b'])[['町丁目名','世代','人口']]
mina_finedata = pd.merge(mina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

mina_finedata['区']='港'

In [12]:
#mina_finedata.to_excel(prefix + dataname + '（港）.xlsx', index=False)

## 5. 渋谷

In [13]:
shib_data = pd.read_excel(prefix + '渋谷区2022年1月_rev.xlsx')

shib_finedata = shib_data[['町丁名','年齢','人口']].rename(columns={'町丁名':'町丁目名'})

shib_finedata = pd.merge(shib_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

shib_finedata['区']='渋谷'

In [14]:
#shib_finedata.to_excel(prefix + dataname + '（渋谷）.xlsx', index=False)

### 台東

### データ統合

In [15]:
finedata = pd.concat([ohta_finedata,meg_finedata,shib_finedata,shina_finedata]).reset_index(drop=True)

finedata['世代'] = ''

# fiveband = list(mina_finedata.loc[:20,:]['世代'])
#print(fiveband)

# ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
# '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
# '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳'
# '105~109歳', '110~114歳', '115~119歳', '120歳以上']

# 0~130まで5づつでビンを作成
bin_band =  list(range(0,131,5))
# binのラベル
fiveband = ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
 '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
 '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳',
 '105~109歳', '110~114歳', '115~119歳', '120~124歳', '125~129歳']#, '130歳以上']

fiveband_df = pd.DataFrame(
            {
                'gord' : list(range(0,26)),
                '世代': fiveband
            }
)


finedatas = finedata[finedata['人口']>0]

finedata['世代']=pd.cut(finedata['年齢'], bin_band, right=False, labels=fiveband)

finedata = pd.merge(finedata, fiveband_df, on='世代', how='left')


In [16]:
print(finedata.info())
print(finedatas.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67589 entries, 0 to 67588
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   町丁目名      67589 non-null  object
 1   年齢        67589 non-null  int64 
 2   人口        67589 non-null  object
 3   市区町村コード   67589 non-null  int64 
 4   市区町村名     67589 non-null  object
 5   大字町丁目コード  67589 non-null  object
 6   大字町丁目名    67589 non-null  object
 7   区         67589 non-null  object
 8   世代        67589 non-null  object
 9   gord      67589 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 5.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 55786 entries, 0 to 67455
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   町丁目名      55786 non-null  object
 1   年齢        55786 non-null  int64 
 2   人口        55786 non-null  object
 3   市区町村コード   55786 non-null  int64 
 4   市区町村名     55786 non-null  object
 5   大

In [17]:
len(fiveband)

26

In [18]:
fd_grp = ['区','町丁目名','大字町丁目コード','世代','gord']
srt = ['大字町丁目コード','gord']
ag=pd.DataFrame(finedata.groupby(fd_grp)['人口'].agg(sum)).sort_values(srt)

In [19]:
ag.reset_index(drop=False).loc[3000:3020,:]

,区,町丁目名,大字町丁目コード,世代,gord,人口
3000,品川,荏原６丁目,131090025006,60~64歳,12,156
3001,品川,荏原６丁目,131090025006,65~69歳,13,131
3002,品川,荏原６丁目,131090025006,70~74歳,14,156
3003,品川,荏原６丁目,131090025006,75~79歳,15,125
3004,品川,荏原６丁目,131090025006,80~84歳,16,103
3005,品川,荏原６丁目,131090025006,85~89歳,17,62
3006,品川,荏原６丁目,131090025006,90~94歳,18,39
3007,品川,荏原６丁目,131090025006,95~99歳,19,12
3008,品川,荏原６丁目,131090025006,100~104歳,20,2
3009,品川,荏原６丁目,131090025006,105~109歳,21,0


In [20]:
c_code.sort_values('大字町丁目コード').reset_index(drop=True).loc[3000:3020,:]

,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名
3000,13123,江戸川区,131230010003,上一色３丁目
3001,13123,江戸川区,131230011001,北葛西１丁目
3002,13123,江戸川区,131230011002,北葛西２丁目
3003,13123,江戸川区,131230011003,北葛西３丁目
3004,13123,江戸川区,131230011004,北葛西４丁目
3005,13123,江戸川区,131230011005,北葛西５丁目
3006,13123,江戸川区,131230012001,西葛西１丁目
3007,13123,江戸川区,131230012002,西葛西２丁目
3008,13123,江戸川区,131230012003,西葛西３丁目
3009,13123,江戸川区,131230012004,西葛西４丁目
